## Imports and libraries installation

In [1]:
%%capture
import os
import cv2 
import ast
import time
import json
import glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader

# install libraries
!pip install wandb -qqq
import wandb
!pip install -U git+https://github.com/qubvel/segmentation_models.pytorch
import segmentation_models_pytorch as smp

## Install custom scripts

In [ ]:
!git clone https://github.com/AGiannoutsos/End-to-End-Lane-Detection
%cd End-to-End-Lane-Detection

## Weights and biases initialization

In [3]:
import wandb
os.environ["WANDB_ENTITY"]  = "andreas_giannoutsos"
os.environ["WANDB_PROJECT"] = "lane_detection"
os.environ["WANDB_RESUME"]  = "allow"
from scripts.wandb_util import *

## Dataset api from the stored files in the hardrive to arrays with label images

In [4]:
from scripts.dataset import *

## Fuctions that store a video clip from the given images

In [5]:
from scripts.visualization import *

## Classic Line Detector

In [6]:
from scripts.classic_line_detector import *

## Download, load, transform and save data images (done once)

In [ ]:
Download TUsimple dataset
download_TUsimple_dataset("test")

# remove "\" from the paths on the json fine
import fileinput
with fileinput.FileInput("test_set/test_tasks_0627.json", inplace=True, backup='.bak') as file:
    for line in file:
        print(line.replace('\\', ""), end='')

# Load and transform images and labels for classic lane detector
test_dataset = Dataset("test_set", (128,128), 10000)

## Classic detector results

## Set the detectors

In [80]:
detectors = [Classic_Line_Detector(10, 50, 7),
             Classic_Line_Detector(10, 100, 7),
             Classic_Line_Detector(10, 150, 7),
             
             Classic_Line_Detector(30, 50, 7),
             Classic_Line_Detector(30, 100, 7),
             Classic_Line_Detector(30, 150, 7),
             
             Classic_Line_Detector(50, 50, 7),
             Classic_Line_Detector(50, 100, 7),
             Classic_Line_Detector(50, 150, 7)]

texts = [detect.title for detect in detectors]

## Create grid videos

In [85]:
%%capture
np.seterr(all="ignore")
grid_video_detector_creator("video_canny_grid_gk7",
                  detectors,
                  texts,
                  test_dataset, 
                  0, 
                  300,
                  grid=(3,3), 
                  labels=False, 
                  fps=24, 
                  overlay_opacity=0.2)

## Create grid images

In [84]:
%%capture
np.seterr(all="ignore")
for i in range(10):
    grid_image_detector_creator("image"+str(i)+"_canny_grid_gk5.jpg",
                  detectors,
                  texts,
                  test_dataset, 
                  np.random.randint(len(test_dataset)), 
                  grid=(3,3), 
                  labels=False, 
                  fps=24, 
                  overlay_opacity=0.2)

In [ ]:
from moviepy.editor import *
path = "video_canny_grid_gk7.avi" 
clip=VideoFileClip(path)
clip.ipython_display(width=512)